### RAG Pipelines- Data Ingestion to Vector DB Pipeline

In [2]:
import os
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

/Users/yashraj/Desktop/RAG/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 4 PDF files to process

Processing: Yashraj_Resume_Nov_2025.pdf
  ✓ Loaded 1 pages

Processing: Yashraj_Ghosalkar_Jan.pdf
  ✓ Loaded 1 pages

Processing: GEOT.pdf
  ✓ Loaded 12 pages

Processing: Yashraj_MMR_Anniversary_Speech_Final.pdf
  ✓ Loaded 4 pages

Total documents loaded: 18


In [4]:
all_pdf_documents


[Document(metadata={'producer': 'pdfTeX-1.40.27', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-12-15T08:03:20+00:00', 'author': '', 'keywords': '', 'moddate': '2025-12-15T08:03:20+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.27 (TeX Live 2025) kpathsea version 6.4.1', 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/pdf/Yashraj_Resume_Nov_2025.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Yashraj_Resume_Nov_2025.pdf', 'file_type': 'pdf'}, page_content='Yashraj Dagadu GhosalkarEmail: yashrajjghosalkar@gmail.com\nLinkedIn|GitHub|mmrstatistics.com Mobile: +91-8010881394\nSummary\nFull-stack engineer who built and operates a live SaaS (MMR Statistics) end-to-end—architecture→payments→\nSEO—on Next.js/TypeScript/A WS. Sole web dev at MMR; led 4 interns, ship production features fast, and keep\nquality high.\nSkills\n•Programming: JavaScript (ES6+), TypeScript, Python, PHP (WordPress), SQL, HTML5, CSS3\n•F ront

In [5]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs


In [6]:
chunks=split_documents(all_pdf_documents)
chunks

Split 18 documents into 108 chunks

Example chunk:
Content: Yashraj Dagadu GhosalkarEmail: yashrajjghosalkar@gmail.com
LinkedIn|GitHub|mmrstatistics.com Mobile: +91-8010881394
Summary
Full-stack engineer who built and operates a live SaaS (MMR Statistics) end-...
Metadata: {'producer': 'pdfTeX-1.40.27', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-12-15T08:03:20+00:00', 'author': '', 'keywords': '', 'moddate': '2025-12-15T08:03:20+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.27 (TeX Live 2025) kpathsea version 6.4.1', 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/pdf/Yashraj_Resume_Nov_2025.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Yashraj_Resume_Nov_2025.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'pdfTeX-1.40.27', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-12-15T08:03:20+00:00', 'author': '', 'keywords': '', 'moddate': '2025-12-15T08:03:20+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.27 (TeX Live 2025) kpathsea version 6.4.1', 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/pdf/Yashraj_Resume_Nov_2025.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Yashraj_Resume_Nov_2025.pdf', 'file_type': 'pdf'}, page_content='Yashraj Dagadu GhosalkarEmail: yashrajjghosalkar@gmail.com\nLinkedIn|GitHub|mmrstatistics.com Mobile: +91-8010881394\nSummary\nFull-stack engineer who built and operates a live SaaS (MMR Statistics) end-to-end—architecture→payments→\nSEO—on Next.js/TypeScript/A WS. Sole web dev at MMR; led 4 interns, ship production features fast, and keep\nquality high.\nSkills\n•Programming: JavaScript (ES6+), TypeScript, Python, PHP (WordPress), SQL, HTML5, CSS3\n•F ront

In [7]:
### convert the text to emveddings
texts=[doc.page_content for doc in chunks]
texts   

['Yashraj Dagadu GhosalkarEmail: yashrajjghosalkar@gmail.com\nLinkedIn|GitHub|mmrstatistics.com Mobile: +91-8010881394\nSummary\nFull-stack engineer who built and operates a live SaaS (MMR Statistics) end-to-end—architecture→payments→\nSEO—on Next.js/TypeScript/A WS. Sole web dev at MMR; led 4 interns, ship production features fast, and keep\nquality high.\nSkills\n•Programming: JavaScript (ES6+), TypeScript, Python, PHP (WordPress), SQL, HTML5, CSS3\n•F rontend: React, Next.js (App Router, RSC), Redux, Tailwind CSS, shadcn/ui\n•Backend & APIs: Node.js, Express.js, REST, Webhooks, JWT/Auth, RBAC, Cron Jobs\n•Auth & Access: NextAuth (Google/OAuth & credentials), role/permission middleware, route protection\n•Payments: HDFC SmartGateway/Juspay flows, checkout orchestration, order lifecycles\n•Data: MongoDB (Mongoose models, indexes), MySQL\n•Cloud/DevOps: AWS (EC2, S3, CloudFront), Docker, CI/CD (GitHub Actions), Linux, Cloudflare Turnstile',
 '•Data: MongoDB (Mongoose models, indexes), 

### embedding And vectorStoreDB

In [8]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager


Loading embedding model: all-MiniLM-L6-v2


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 855.95it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model loaded successfully. Embedding dimension: 384


### VectorStore

In [10]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore
    

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 142


In [11]:
chunks

[Document(metadata={'producer': 'pdfTeX-1.40.27', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-12-15T08:03:20+00:00', 'author': '', 'keywords': '', 'moddate': '2025-12-15T08:03:20+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.27 (TeX Live 2025) kpathsea version 6.4.1', 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/pdf/Yashraj_Resume_Nov_2025.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Yashraj_Resume_Nov_2025.pdf', 'file_type': 'pdf'}, page_content='Yashraj Dagadu GhosalkarEmail: yashrajjghosalkar@gmail.com\nLinkedIn|GitHub|mmrstatistics.com Mobile: +91-8010881394\nSummary\nFull-stack engineer who built and operates a live SaaS (MMR Statistics) end-to-end—architecture→payments→\nSEO—on Next.js/TypeScript/A WS. Sole web dev at MMR; led 4 interns, ship production features fast, and keep\nquality high.\nSkills\n•Programming: JavaScript (ES6+), TypeScript, Python, PHP (WordPress), SQL, HTML5, CSS3\n•F ront

In [12]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 108 texts...


Batches: 100%|██████████| 4/4 [00:01<00:00,  2.03it/s]

Generated embeddings with shape: (108, 384)
Adding 108 documents to vector store...
Successfully added 108 documents to vector store
Total documents in collection: 250


### Retriever Pipeline From VectorStore

In [13]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)



In [14]:
rag_retriever

In [15]:
rag_retriever.retrieve("Generative Engine Optimization")

Retrieving documents for query: 'Generative Engine Optimization'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.69s/it]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_e586e242_99',
  'content': 'with statistical deviations in Table 6\nC.1 GEO in the Wild : Experiments with\nDeployed Generative Engine\nWe also evaluate our proposed Generative Engine Optimization\nmethods on real-world deployed Generative Engine: Perplexity.ai.\nSince perplexity.ai does not allow the user to specify source URLs,\nwe instead provide source text as file uploads to perplexity.ai while\nensuring all answers are generated only using the file sources pro-\nvided. We evaluate all our methods on a subset of 200 samples of\nour test set. Results using Perplexity.ai are shown in Table 7.',
  'metadata': {'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
   'total_pages': 12,
   'trapped': '/False',
   'creationdate': '2024-07-01T00:45:18+00:00',
   'source_file': 'GEOT.pdf',
   'producer': 'pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
   'source': '../data/pdf/GEOT.pdf',


In [16]:
rag_retriever.retrieve("GEO Methods")

Retrieving documents for query: 'GEO Methods'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.79it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


[{'id': 'doc_fb462849_61',
  'content': 'best combination (Fluency Optimization and Statistics Addition)\noutperforming any single GEO strategy by more than 5.5%4. Fur-\nthermore, Cite Sources significantly boosts performance when used\n4Due to cost constraints, the analysis was conducted on a subset of 200 examples from\nthe test split, and therefore the numbers presented here differ from those in Table 1\nin conjunction with other methods (Average: 31.4%), despite it being\nrelatively less effective when used alone (8% lower than Quotation\nAddition). The findings underscore the importance of studyingGEO\nmethods in combination, as they are likely to be used by content\ncreators in the real world.\n5.4 Qualitative Analysis\nWe present a qualitative analysis of GEO methods in Table 4, con-\ntaining representative examples where GEO methods boost source\nvisibility with minimal changes. Each method optimizes a source\nthrough suitable text additions and deletions. In the first example,